## :rocket: Simplifying Financial Analysis with Streamlit in Snowflake: Creating Candlestick Charts Made Easy! :bar_chart:

### How can you quickly understand a particular asset's price movements and trends?

Candlestick Charts are Powerful Visualizations Used by Traders and Investors.:chart_with_upwards_trend:

### What is a Candlestick Chart?

A candlestick chart is a powerful visual tool that shows four crucial price points (OHLC):
- Open: Price at market open
- High: Highest price during the period
- Low: Lowest price during the period
- Close: Price at market close

### Reading the Candle:
- :large_green_circle: Green/White Candle: Closing price > Opening price (Bullish)
- :red_circle: Red/Black Candle: Closing price < Opening price (Bearish)
- The "wick" or "shadow" shows the high and low prices!

### Why it matters:
- Instantly visualize price trends :fire:
- Identify market sentiment :ox: :bear: :grinning: :unamused:
- Spot potential reversal patterns :mag:
- Make informed trading decisions :thinking_face:

## Create Candlestick Charts :chart_with_upwards_trend: in Snowflake :snowflake: 
### Author: [Prasanna Rajagopal](https://www.linkedin.com/in/prasannarajagopal/)
### Load the data to create Candlestick charts in a Snowflake Notebook. 
- Follow the instructions in this Snowflake documentation to create a table and load data into it.  
    - [Load Data Using Snowsight](https://docs.snowflake.com/en/user-guide/data-load-web-ui#load-data-using-sf-web-interface)
    - Load this file - [Candlestick_Chart_Demo_Data.csv](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Company_Financials/Candlestick_Chart/Candlestick_Chart_Demo_Data.csv)
    - Name the table: COMPANY_STOCK_PRICES_OHLC_TBL

(Data Source: [Polygon.io](https://polygon.io/))

Created :calendar:: March 2025

## Test the COMPANY_STOCK_PRICES_OHLC_TBL Table

In [ ]:
SELECT * FROM COMPANY_STOCK_PRICES_OHLC_TBL LIMIT 5;

## Ensure that Plotly Python Package is Added to the Notebook.  
### Please follow instructions in this Snowflake documentation
### [Snowflake Notebooks Import Packages](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-import-packages)  

## Import the Python modules into the Snowflake Notebook
### Get an active session to your Snowflake database

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
import plotly.graph_objs as go

# Establish Snowflake session
session = get_active_session()

## Fetch the Distinct Ticker Symbols From the Table

In [ ]:
selTickerSymbol = """
SELECT DISTINCT TICKER_SYMBOL
FROM COMPANY_STOCK_PRICES_OHLC_TBL
""";

# Execute the SQL query and get the DataFrame
df = session.sql(selTickerSymbol).collect()

# Use the list of ticker symbols in the selectbox
selected_ticker = st.selectbox(
    "Select a Ticker Symbol for Constructing the Candlestick Chart:",
    df,
    key="selectbox_passing_df"
)

st.write(f"You selected: {selected_ticker}") # Example of using the selected value


# An Alternative method instead of passing the the Snowpark Dataframe to the Selectbox
# Extract the ticker symbols into a list
ticker_symbols = [row['TICKER_SYMBOL'] for row in df]

# Use the list of ticker symbols in the selectbox
selected_ticker1 = st.selectbox(
    "Select a Ticker Symbol for Constructing the Candlestick Chart:",
    ticker_symbols,
    key="selectbox_passing_List"
)

## Fetch the OHLC Data for the Selected Ticker Symbol

In [ ]:
# Fetch and filter data directly into a Pandas DataFrame
OHLC_pd = session.table("COMPANY_STOCK_PRICES_OHLC_TBL").filter(col("TICKER_SYMBOL") == selected_ticker).to_pandas()
OHLC_pd.head()

## Create a Candlestick Chart Using Plotly

In [ ]:
# Create and display Candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=OHLC_pd['OHLC_DATE'],
    open=OHLC_pd['OPEN_PRICE'],
    high=OHLC_pd['HIGH_PRICE'],
    low=OHLC_pd['LOW_PRICE'],
    close=OHLC_pd['CLOSE_PRICE'],
    name="Candlesticks",
)])

fig.update_layout(
    title= selected_ticker,
    yaxis_title="Price ($)",
    xaxis_rangeslider_visible=False,
    plot_bgcolor="white",
    width=700,
    height=500
).update_yaxes(tickprefix="$") # Chaining update_yaxes.

st.write(fig)